In [148]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from joblib import dump

In [149]:
# For reproducibility.
rng = np.random.RandomState(1)

# Dataset

In [150]:
df = pd.read_csv('data/augmented_dataset.csv', index_col='id')

features = [i for i in df.columns if i != 'class']
target = 'class'

X, y = df.loc[:, features], df.loc[:, target]

In [151]:
df.head(3)

,ANDN920101,ARGP820101,ARGP820102,ARGP820103,BEGF750101,BEGF750102,BEGF750103,BHAR880101,BIGC670101,BIOV880101,...,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi,class
id,,,,,,,,,,,,,,,,,,,,,
10gs,-0.117565,0.164055,-0.386466,-0.524996,-0.655534,-0.310020,0.380819,0.619936,0.361497,-0.406576,...,-0.001390,-0.155634,-0.155634,-0.291240,-0.271137,0.118107,0.286221,0.335830,0.278359,1
11gs,-0.102087,0.072364,-0.433112,-0.522733,-1.105537,-0.766021,0.940460,1.220417,0.111012,-0.628022,...,0.183152,-0.016953,-0.016953,-0.232940,-0.122073,0.596794,0.680634,0.736305,0.819202,1
13gs,-1.386729,2.196977,0.058786,0.142759,-0.968929,1.071801,-0.915684,0.522422,0.520896,1.053327,...,-0.365395,-0.512734,-0.512734,-0.596358,-0.479828,-0.291246,-0.312462,-0.317647,-0.368448,1


# Training

## Configurations

All the configurations have the form:

$$ \text{scaler} \mapsto \text{reducer} \mapsto \text{classifier}$$

In [152]:
steps = [
    ('scaler', StandardScaler()),
    ('reducer', PCA(random_state=rng)),
    ('classifier', DummyClassifier()),
]

param_grid = [
    {
        'reducer__n_components': [20, 50, 100, 300],
        'classifier': [RandomForestClassifier(random_state=rng)],
        'classifier__n_estimators': [100, 200, 500],
        'classifier__min_samples_split': [2, 5, 10],
    },
    {
        'reducer__n_components': [20, 50, 100, 300],
        'classifier': [XGBClassifier(random_state=rng)],
        'classifier__n_estimators': [50, 100, 500, 1000, 2000],
        'classifier__learning_rate': [0.1, 0.5, 1.],
        'classifier__max_depth': [2, 4, 6, 8],
    },
]

## CV configuration

In [153]:
# Create the pipeline: scaler => reducer => classifier.
pipeline = Pipeline(steps=steps)

# Spli the data into train and test with stratification.
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, train_size=0.9, random_state=1)

# 5-fold cross validation for tuning hyperparameters.
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# Use AUC as validation metric.
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kf, scoring='roc_auc', n_jobs=-1, refit=True)

## Tuning hyperparameters and refitting on the whole training set

In [154]:
grid.fit(X_train, y_train)

/home/ansar/venvir/ml/lib64/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('reducer',
                                        PCA(random_state=RandomState(MT19937) at 0x7FDBBF0C6640)),
                                       ('classifier', DummyClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [RandomForestClassifier(random_state=RandomState(MT19937) at 0x7FDBBF0C6640)],
                          'classifie...
                                                       max_leaves=None,
                                                       min_child_weight=None,
                                                       missing=nan,
                                                       monotone_constraints=None,
                                                       multi_strategy=None,
                                                       n_estimators=None,
                                                       n_jobs=None,
                                                       num_parallel_tree=None,
                                                       random_state=1484172013, ...)],
                          'classifier__learning_rate': [0.1, 0.5, 1.0],
                          'classifier__max_depth': [2, 4, 6, 8],
                          'classifier__n_estimators': [50, 100, 500, 1000,
                                                       2000],
                          'reducer__n_components': [20, 50, 100, 300]}],
             scoring='roc_auc')

# Save the best model and the grid

In [155]:
best_model = grid.best_estimator_
best_model

Pipeline(steps=[('scaler', StandardScaler()),
                ('reducer',
                 PCA(n_components=50,
                     random_state=RandomState(MT19937) at 0x7FDBBF0C6140)),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               fea...olicy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=2000, n_jobs=None,
                               num_parallel_tree=None, random_state=1791095845, ...))])

In [156]:
dump(best_model, 'saved_models/best_model.joblib')
dump(grid, 'saved_models/gridsearch_cv.joblib')

['saved_models/gridsearch_cv.joblib']